TODO: 
1. Load and restructure data. 
2. Restructure sentence boundaries. 
3. Run Phrases model 
4. Run CBOW model 
5. Assess CBOW model

# 0. Load and reformat data

In [3]:
import spacy

In [2]:
! pip install gensim

  Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl.metadata (8.2 kB)
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp311-cp311-win_amd64.whl (24.0 MB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl (46.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


In [4]:
import gensim

In [8]:
import json 
import gzip
import numpy as np

path = r"C:\\Users\bened\DataScience\ANLP\AT2\\preprocessed_data\screenplays_preprocessed2.json.gz"

with gzip.open(path, 'rt', encoding='utf-8') as f:
    data = json.load(f)

print(type(data))

<class 'dict'>


In [9]:
print(len(data))
print(data["0"][:10])

1995
['@1', 'night', 'write', 'steve', 'koren', 'chris', 'kattan', 'june', 'panoramic', 'view']


In [10]:
screenplay_data = {int(k): v for k, v in data.items()}

In [11]:
print(len(screenplay_data))

1995


In [51]:
def measure_vocab(data_dict):    
    unique_tokens = set(
            token for tokens in data_dict.values() for token in tokens
        )
    vocab_size = len(unique_tokens)
    print("Size of vocab:", vocab_size)

In [52]:
data_vocab = measure_vocab(screenplay_data)
print(data_vocab)

217196


In [60]:
# check for data corruption 
import re 

allowed = ['@', '.', ':', '\n', '-']

def find_nonword_chars(data_dict):
    filtered_tokens = set()
    for value in data_dict.values():
        for token in value:
            if not isinstance(token, str) or not token:
                continue
            else:
                if any(char for char in token if re.search(r'\W', char) and char not in allowed):
                    filtered_tokens.add(token)
    if not filtered_tokens:
        print("No nonword chars remaining")
        return None
    else:
        return filtered_tokens

In [61]:
nonwords = find_nonword_chars(screenplay_data)
print(nonwords)

{"fuckin'-ben", "d'er", "m'sieur", '\\10f', 'set/', '\x00s\x00e\x00e\x00.\x00.\x00.\x00', "phey'ra", '\x00s\x00p\x00o\x00k\x00e\x00', '\x00n\x00o\x00t\x00', 'int.-ext./day', '\x00s\x00t\x00a\x00r\x00i\x00n\x00g\x00', '\x00c\x00a\x00r\x00n\x00a\x00t\x00i\x00o\x00n\x00', "t'aime", 'sorry-~looke', '-+-standing', '\x00n\x00o\x00n\x00c\x00h\x00a\x00l\x00a\x00n\x00c\x00e\x00.\x00', '\x00s\x00e\x00n\x00d\x00', 'â\x80\x9cshowerâ\x80\x9d', '1/2h', 'roof\x97', "i'l1", 'aquarium\x94', '\x00b\x00e\x00a\x00r\x00d\x00', "feelin'ts", '\x00p\x00a\x00s\x00t\x00', 'x¥ou', 'direc=', 't¢', 'be-~', '¿', "l'ile", '\x00t\x00e\x00l\x00e\x00p\x00h\x00o\x00n\x00e\x00d\x00', "t'keep", 'â\x80\x98em', '\x93nah', "c'mcn", "i'll-", '\x00d\x00a\x00n\x00g\x00l\x00e\x00s\x00', "blow'em", '\x00g\x00l\x00a\x00s\x00s\x00', 'am|', '2,500', '\x00t\x00i\x00n\x00', 'arabi¢', "it's.good", 'me\x97', "pers'nal", '\x00t\x00w\x00o\x00-\x00c\x00o\x00l\x00o\x00r\x00', 'knowâ\x80¢', '\x00t\x00h\x00u\x00m\x00b\x00', '\x00c\x00-\x002\x

In [55]:
# clean up again! 
hex_pat = re.compile(r'[\x00-\x1F\x7F-\x9F]')

def clean_corruptions(string):

    ## REMOVING CHARS
    # remove any hexademical placeholders
    string = re.sub(hex_pat, '', string)
    # remove /
    string = re.sub("/", "", string)
    # remove |
    string = re.sub(r"|", '', string)
    # remove , 
    string = re.sub(r",", '', string)
    # remove ~
    string = re.sub("~", '', string)

    ## REPLACING CHARS
    # replace ' with space
    string = re.sub("'", ' ', string)
    # replace cent with c
    string = re.sub(r'¢', 'c', string)
    # replace - with space
    string = re.sub("-", " ", string)
    # if 2 word chars are separated by anything that's not a space or another word char, sub with a space
    string = re.sub(r'(\w)[^\w\s](\w)', r'\1 \2', string)

    # strip out any trailing whitespace
    string = string.strip()
    
    # return the string if it's not empty
    if string:
        return string

In [57]:
filtered_data = {}

for key, value in screenplay_data.items():
    filtered_data[key] = [clean_corruptions(token) for token in value]

In [59]:
print(type(filtered_data))
print(len(filtered_data))

<class 'dict'>
1995


In [62]:
nonwords = find_nonword_chars(filtered_data)
print(nonwords)

{'p il', 'build a', 'y bre', '\\10f', 'of f£ handedly', 'abra am', '. uncover', 'chap n', 'out mappe', 'in tho', 'l 36', 'e l', 'm damn', '82e c', 'pal mateer', 'st. george', 'his cheek', 'b 75', 'nu uanu', 'int afternoon', 'party .', '110 the', 'they r', 'o ter', 'jay n justice', 'table he', 'don t want', 't aa', 'direc=', '¿', '10 03am', 'f 14s', 'location href', 'n curl', 'mr angier', 'we tre', '. lou', 'don t have', 'dry heaving', 't 4', 's m.u', 'land office', 'm a.s h', 'am|', 'was n', 'freezer jerry', 'we ve', 'tector and', 'rev 61200', 'm r.i', 'e m.t s', 'theï ½', 'decide i', 'g 1', 'don €', 'b f.a', 'c e.o', 'slides down', 'l ultima', 'wat^.', 'b 47', '. three', 'saiding glaes', 'dant necklace', 'ever body', 'p m.k', '2h8 5', 'its slant', 'gave chase', 'ekin==', 'ber edict', 'mok head', '|a', 'i mgette', 'h o.g', '11 05pm', 'moff it', 'my fost', 'man charged', 'beautiful it', 'giant with', 'hi nice', 'near wire', 'the train', '104a 106', 'vi stness', 'l avrence', 'li characte

In [24]:
filtered_vocab_size = measure_vocab(filtered_data)
print(filtered_vocab_size)

NameError: name 'measure_vocab' is not defined

In [25]:
print(screenplay_data[10][:10])

['@0', 'last', 'mohicans', 'write', 'michael', 'mann', 'christopher', 'crowe', '\n ', 'screen']


In [26]:
import copy 

mo = copy.deepcopy(screenplay_data[10])
for i, tok in enumerate(mo):
    if i == 50:
        break
    else:
        print(tok)

@0
last
mohicans
write
michael
mann
christopher
crowe

 
screen
leaf
crystal
drop
precipitation
landscape
miniature
hear
forest
distant
bird
sound
seem
piece
sunlight
refract
drop
water
paint
patch
moss
yellow
whisper
wind
join
sound
mix
distant
rustling
get
close
louder
shallow
breathing
get
ominous
interloper
floor
forest
come
suddenly


In [27]:
for t in mo:
    if t.startswith("@"):
        print(t)

@0
@1
@2
@1
@1
@0
@1
@1
@1
@1
@1
@1
@2
@1
@1
@0
@1
@2
@2
@1
@1
@2
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@0
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@2
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@2
@1
@2
@1
@2
@1
@2
@2
@2
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@0
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@0
@1
@2
@2
@1
@1
@1
@1
@1
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@0
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@0
@2
@1
@1
@1
@1
@1
@2
@2
@1
@1
@1
@2
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1
@1


In [28]:
# search for @ and \n
counter = 0
for i, t in enumerate(mo):
    if counter == 20:
        break
    if t.startswith("\n"):
        print(mo[i-5:i+5])
        counter += 1

['write', 'michael', 'mann', 'christopher', 'crowe', '\n ', 'screen', 'leaf', 'crystal', 'drop']
['heavy', 'even', 'breathe', 'learn', 'later', '\n ', '@2', ':', 'man', 'unca']
['branch', 'disturb', 'make', 'sound', 'tan', '\n ', 'two', 'hundred', 'fifty', 'yards']
['caliber', 'round', 'program', '.', 'moment', '\n ', '@2', ':', 'impact', '@1']
['long', 'knife', 'approach', 'fall', 'elk', '\n ', '@0', 'exterior', 'interior', 'cameron']
['hand', 'fence', 'chingachgook', 'john', 'cameron', '\n ', 'doorway', 'cameron', 'interior', 'set']
['good', 'another', 'year', 'pass', 'john', '\n ', 'gettin', 'along', 'yes', 'warm']
['heart', 'farm', 'good', 'year', 'john', '\n ', 'good', 'year', 'corn', 'mohawk']
['take', 'much', 'fur', 'horicane', 'sic', '\n ', '@1', 'trap', 'tradin', 'castleton']
['\n ', '@1', 'trap', 'tradin', 'castleton', '\n ', 'schylerville', 'dutch', 'french', 'english']
['mohican', 'child', 'leave', 'brother', 'peace', '\n ', 'laugh', 'hawkeye', 'chingachgook', '\n ']
['peac

I've now loaded my preprocessed data into a new notebook and want to prepare it for a gensim Phrases model. The Phrases module takes lists of word-tokenized sentences as input (I believe?).   Currently my data is structured like this: 
{screenplay_index(int):['@n', 't1', t2', ..., 'ti', '\n'

Where n is a label 0, 1 or 2 that provides information on the line

In [29]:
# search for @ and \n
counter = 0
for i, t in enumerate(mo):
    if counter == 20:
        break
    if t.startswith(r"@") or t.startswith(r"\\n"):
        print(mo[i-1:i+2])
        counter += 1


[]
['foot', '@1', 'rocket']
['\n ', '@2', ':']
['day', '@1', 'rocket']
['tribe', '@1', 'shirt']
['foliage', '@0', 'suddenly']
['shoulder', '@1', 'cock']
['sound', '@1', 'slip']
['sight', '@1', 'five']
['tight', '@1', 'hold']
['sound', '@1', 'crack']
['lightning', '@1', 'leap']
['\n ', '@2', ':']
['impact', '@1', 'three']
['worldliness', '@1', 'anglo']
['\n ', '@0', 'exterior']
['\n ', '@1', 'trap']
['\n ', '@2', ':']
['\n ', '@2', ':']
['fast', '@1', 'laugh']


# Prepare Data for Vectorization

Desire data structure: 
{
    label:
    {sentence_vector:
        {word_vectors}
    }
}

As sentences:
{label: [sentences[sent1][sent2][etc.]]}


In [30]:
# find and return all non-word chars in the data  
import re

allowed = {'@', '.', ':', '\n', '-'}

filtered_tokens = set()

for tokens in screenplay_data.values():
    for token in tokens:
        if any(
            char for char in token if re.search(r'\W', char) and char not in allowed
        ):
            filtered_tokens.add(token)

print(filtered_tokens)

{"fuckin'-ben", "d'er", "m'sieur", '\\10f', 'set/', '\x00s\x00e\x00e\x00.\x00.\x00.\x00', "phey'ra", '\x00s\x00p\x00o\x00k\x00e\x00', '\x00n\x00o\x00t\x00', 'int.-ext./day', '\x00s\x00t\x00a\x00r\x00i\x00n\x00g\x00', '\x00c\x00a\x00r\x00n\x00a\x00t\x00i\x00o\x00n\x00', "t'aime", 'sorry-~looke', '-+-standing', '\x00n\x00o\x00n\x00c\x00h\x00a\x00l\x00a\x00n\x00c\x00e\x00.\x00', '\x00s\x00e\x00n\x00d\x00', 'â\x80\x9cshowerâ\x80\x9d', '1/2h', 'roof\x97', "i'l1", 'aquarium\x94', '\x00b\x00e\x00a\x00r\x00d\x00', "feelin'ts", '\x00p\x00a\x00s\x00t\x00', 'x¥ou', 'direc=', 't¢', 'be-~', '¿', "l'ile", '\x00t\x00e\x00l\x00e\x00p\x00h\x00o\x00n\x00e\x00d\x00', "t'keep", 'â\x80\x98em', '\x93nah', "c'mcn", "i'll-", '\x00d\x00a\x00n\x00g\x00l\x00e\x00s\x00', "blow'em", '\x00g\x00l\x00a\x00s\x00s\x00', 'am|', '2,500', '\x00t\x00i\x00n\x00', 'arabi¢', "it's.good", 'me\x97', "pers'nal", '\x00t\x00w\x00o\x00-\x00c\x00o\x00l\x00o\x00r\x00', 'knowâ\x80¢', '\x00t\x00h\x00u\x00m\x00b\x00', '\x00c\x00-\x002\x

In [31]:
print(len(filtered_tokens))

9483


In [32]:
import numpy as np
import re

def preprocess_for_phrases(data):

    labeled_sentences = {}

    for screenplay_idx, tokens in data.items():

        current_sentence = []
        processed_line = []
        line_label = None

        for token in tokens:
            
            # parse label
            if token.startswith('@'):
                line_label = np.int8(token[1:])
                labeled_sentences[line_label] = []

            # skip if token contains non-word chars 
            elif re.search(r'\W', token) and token not in {'.', '\n'}:
                continue

            elif token == ':':
                continue

            # skip over '-' 
            elif token == '-':
                continue
            
            elif token == '.':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            elif token == '\n':
                if current_sentence:
                    processed_line.append(current_sentence)
                    current_sentence = []
            
            else:
                current_sentence.append(token)
        
        if current_sentence:
            processed_line.append(current_sentence)
        
        if line_label is not None:
            labeled_sentences[line_label].extend(processed_line)
        
    return labeled_sentences


In [33]:
prepared_data = preprocess_for_phrases(screenplay_data)

In [34]:
sents = [sent for sentences in prepared_data.values() for sent in sentences]

In [35]:
unique_tokens = set(token for sentence in sents for token in sentence)
vocabulary_size = len(unique_tokens)
print(vocabulary_size)

2260


# Phrases

In [36]:
# ! pip install --upgrade numpy gensim

In [37]:
# import numpy 
# import gensim
# import scipy

# print(numpy.__version__)
# print(gensim.__version__)
# print(scipy.__version__)

In [38]:
from gensim.models import Word2Vec, phrases

In [39]:
bigrams = phrases.Phrases(
    sentences=sents,
    connector_words=phrases.ENGLISH_CONNECTOR_WORDS
)
bigram_model = phrases.Phraser(bigrams)
bigrams_dict = bigrams.export_phrases()

In [18]:
# ! pip install pandas

  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)


In [41]:
import pandas as pd

bigrams_df = pd.DataFrame.from_dict(
    bigrams_dict, 
    orient='index'
)
bigrams_df.reset_index(inplace=True)
bigrams_df.columns = ["phrase", "score"]
bigrams_df.sort_values('score', inplace=True, ascending=False)
print(bigrams_df.head(10))
print(bigrams_df.tail(10))

          phrase       score
17          8_yr  359.520000
1   baseball_cap  299.600000
2      duct_tape  256.800000
16    big_sister  214.000000
18        yr_old  172.846154
15     towel_boy  136.960000
20      sir_well   88.117647
19      well_sir   88.117647
6    view_mirror   80.250000
12    rental_car   79.377717
               phrase      score
8         living_room  74.900000
4   service_stairwell  74.900000
3          cell_phone  74.900000
5           rear_view  56.647059
11       coffee_table  34.569231
14           car_park  28.087500
13       basement_car  17.554688
9      bedroom_mother  17.368116
10          day_later  12.895265
7        mother_house  10.020067


### Trigrams

Plausible trigrams like "x_day_later", "x_yr_old", so we'll run for trigrams. 

In [42]:
trigrams = phrases.Phrases(bigrams[sents])
# trigram_model = phrases.Phraser(trigrams)
trigrams_dict = trigrams.export_phrases()
trigrams_df = pd.DataFrame.from_dict(
    trigrams_dict, 
    orient='index'
)
trigrams_df.reset_index(inplace=True)
trigrams_df.columns = ["phrase", "score"]
trigrams_df.sort_values('score', inplace=True, ascending=False)
print(trigrams_df.head(10))
print(trigrams_df.tail(10))


               phrase         score
0      goldenrod_page  48981.000000
1        baseball_cap  13512.000000
14           8_yr_old   1441.280000
13          towel_boy    750.666667
10         rental_car    616.336842
5         living_room    563.000000
2   service_stairwell    375.333333
12  basement_car_park    346.461538
4    rear_view_mirror    250.222222
3           rear_view    150.133333
                phrase       score
5          living_room  563.000000
2    service_stairwell  375.333333
12   basement_car_park  346.461538
4     rear_view_mirror  250.222222
3            rear_view  150.133333
11        basement_car  118.526316
9         coffee_table   91.918367
8   goldenrod_page_joe   61.981651
6       bedroom_mother   40.214286
7            day_later   23.397403


coffee_table is the ngram with the lowest score that still makes sense, so I'll run this again and make the cut-off at 90. 

In [49]:
trigrams = phrases.Phrases(bigrams[sents], threshold=90)
# trigram_model = phrases.Phraser(trigrams)
trigrams_dict = trigrams.export_phrases()
trigrams_df = pd.DataFrame.from_dict(
    trigrams_dict, 
    orient='index'
)
trigrams_df.reset_index(inplace=True)
trigrams_df.columns = ["phrase", "score"]
trigrams_df.sort_values('score', inplace=True, ascending=False)
print(trigrams_df.head(10))
print(trigrams_df.tail(10))

               phrase         score
0      goldenrod_page  48981.000000
1        baseball_cap  13512.000000
11           8_yr_old   1441.280000
10          towel_boy    750.666667
7          rental_car    616.336842
5         living_room    563.000000
2   service_stairwell    375.333333
9   basement_car_park    346.461538
4    rear_view_mirror    250.222222
3           rear_view    150.133333
               phrase        score
11           8_yr_old  1441.280000
10          towel_boy   750.666667
7          rental_car   616.336842
5         living_room   563.000000
2   service_stairwell   375.333333
9   basement_car_park   346.461538
4    rear_view_mirror   250.222222
3           rear_view   150.133333
8        basement_car   118.526316
6        coffee_table    91.918367


In [43]:
folder_path = r"C:\\Users\\bened\\DataScience\\ANLP\\AT2\\models"
trigram_model.save(f"{folder_path}\\trigram_model.pkl")

In [44]:
cbow_model = Word2Vec(
    sentences=trigrams[sents],
    vector_size=64,
    window=8,
    sg=0
)

In [45]:
word_vectors = cbow_model.wv 
print(len(word_vectors))
# word_vectors.save(f"{folder_path}\\cbow3_vectors.wordvectors")

360


# Vector Evaluation

In [46]:
def find_synonyms(vectors, word, topn=5):
    synonyms = vectors.most_similar(word, topn=topn)
    print(f"Words most similar to {word}:")
    for w, s in synonyms:
        print(f'{w}: {s:.4f}')

In [47]:
print(len(word_vectors))

360


In [48]:
find_synonyms(word_vectors, 'fuck', 5)

KeyError: "Key 'fuck' not present in vocabulary"